# Fine-tune YOLO for Emergency Vehicle

## 🚀 Key Features

* **Dataset Sourcing**: Guidance on where to find and collect images of emergency vehicles.
* **Data Annotation**: Steps for annotating the dataset with segmentation masks using an external tool.
* **Format Conversion**: Conversion of standard COCO JSON annotations into the YOLO segmentation format.
* **Model Training**: Fine-tuning a pre-trained YOLOv11 segmentation model on the custom ambulance dataset.
* **Video Inference**: Applying the trained model to a video to detect and segment emergency vehicles in real-time.


## 📚 Libraries & Prerequisites

* **Core Libraries**: `ultralytics`.
* **Environment**: A Python environment with GPU support (like Google Colab) is highly recommended for efficient model training.
* **Dataset**: A custom-collected dataset of emergency vehicle images, an annotation file in COCO format, and test videos for inference.


# Load dataset

In [1]:
link = 'https://www.kaggle.com/datasets/aliwannous2021/ambulance-vehicles?select=ambulance2+%28101%29.jpg'

In [ ]:
# !pip install git+https://github.com/Labellerr/SDKPython.git
# !pip install ipyfilechooser
# !git clone https://github.com/Labellerr/yolo_finetune_utils.git

In [2]:
# all the imports required for this notebook
from labellerr.client import LabellerrClient
from labellerr.core.datasets import create_dataset_from_local
from labellerr.core.annotation_templates import create_template
from labellerr.core.projects import create_project
from labellerr.core.schemas import DatasetConfig, AnnotationQuestion, QuestionType, CreateTemplateParams, DatasetDataType, CreateProjectParams, RotationConfig
from labellerr.core.projects import LabellerrProject
from labellerr.core.exceptions import LabellerrError

import uuid
from ipyfilechooser import FileChooser

In [3]:
api_key = input("YOUR_API_KEY")        # go to labellerr workspace to get your API key
api_secret = input("YOUR_API_SECRET")  # go to labellerr workspace to get your API secret
client_id = input("YOUR_CLIENT_ID")   # Contact labellerr support to get your client ID i.e. support@tensormatics.com

client = LabellerrClient(api_key, api_secret, client_id)

### create a dataset on labeller from local folder

In [8]:
# Create a folder chooser starting from a directory (for example, your home directory)
chooser = FileChooser('data')

# Set the chooser to folder selection mode only
chooser.title = 'Select a folder containing your dataset'
chooser.show_only_dirs = True

# Display the widget
display(chooser)

FileChooser(path='/Users/phoenix/PythonProject/hands-on computer-vision/fine-tuning/data', filename='', title=…

In [9]:
path_to_dataset = chooser.selected_path
print("You selected:", path_to_dataset)

You selected: /Users/phoenix/PythonProject/hands-on computer-vision/fine-tuning/data


In [11]:
my_dataset_type = input("Enter your dataset type (video or image): ").lower()
print("Selected dataset type:", my_dataset_type)

Selected dataset type: video


In [ ]:
dataset = create_dataset_from_local(
    client=client,
    dataset_config=DatasetConfig(dataset_name="My Dataset", data_type="image"),
    folder_to_upload=path_to_dataset
)

print(f"Dataset created with ID: {dataset.dataset_id}")

### create annotation project on labeller

In [ ]:
template = create_template(
    client=client,
    params=CreateTemplateParams(
        template_name="My Template",
        data_type=DatasetDataType.image,
        questions=[
            AnnotationQuestion(
                question_number=1,
                question="Object",
                question_id=str(uuid.uuid4()),
                question_type=QuestionType.polygon,
                required=True,
                color="#FF0000"
            )
        ]
    )
)
print(f"Annotation template created with ID: {template.annotation_template_id}")

In [ ]:
dataset.status()        # wait until dataset is processed before creating project

project = create_project(
    client=client,
    params=CreateProjectParams(
        project_name="My Project",
        data_type=DatasetDataType.image,
        rotations=RotationConfig(
            annotation_rotation_count=1,
            review_rotation_count=1,
            client_review_rotation_count=1
        )
    ),
    datasets=[dataset],
    annotation_template=template
)

print(f"✓ Project created: {project.project_id}")

### exxport annotation

## convert COCO JSON Annotation to YOLO format

In [ ]:
# Clone the utility repository to access the required functions
!git clone https://github.com/yashsuman15/yolo_finetune_utils.git

In [ ]:
from yolo_finetune_utils.coco_yolo_converter.seg_converter import coco_to_yolo_converter

result = coco_to_yolo_converter(
            json_path='./annotation.json',
            images_dir='./dataset',
            output_dir='yolo_format'
            )

## fine-tuning with Ultralytics Package

In [ ]:
# !pip install ultralytics

In [ ]:
import ultralytics
ultralytics.checks()

In [ ]:
from ultralytics import YOLO
from IPython.display import Image

In [ ]:
location = !pwd
dataset_path = f"{location[0]}/yolo_format"
print(f"Dataset path: {dataset_path}")

### train YOLO11 Model on a Custom Dataset

In [ ]:
!yolo task=segment mode=train data={dataset_path}/data.yaml model="yolo11x-seg.pt" epochs=200 imgsz=640 batch=10

### inference with Fine-tune on Videos

In [ ]:
!yolo task=segment mode=predict model="./runs/segment/train/weights/best.pt" conf=0.7 source="./video/g.mp4" save=True show_labels=False